In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
**Exploração e análise de dados de crédito com SQL**

_Os dados:_
Os dados representam informações de clientes de um banco e contam com as seguintes colunas:

idade = idade do cliente
sexo = sexo do cliente (F ou M)
dependentes = número de dependentes do cliente
escolaridade = nível de escolaridade do clientes
salario_anual = faixa salarial do cliente
tipo_cartao = tipo de cartao do cliente
qtd_produtos = quantidade de produtos comprados nos últimos 12 meses
iteracoes_12m = quantidade de iterações/transacoes nos ultimos 12 meses
meses_inativo_12m = quantidade de meses que o cliente ficou inativo
limite_credito = limite de credito do cliente
valor_transacoes_12m = valor das transações dos ultimos 12 meses
qtd_transacoes_12m = quantidade de transacoes dos ultimos 12 meses
A tabela foi criada no AWS Athena junto com o S3 Bucket com uma versão dos dados disponibilizados em: https://github.com/andre-marcos-perez/ebac-course-utils/tree/main/dataset

_Exploração de dados:_
A primeira fase da análise é entender o que tem na nossa matéria prima. Vamos a exploração de dados:

Qual a quantidade de informações temos na nossa base de dados?

Query: **_SELECT count(*) FROM credito_**

Reposta: 2564 linhas

ps.: A base de dados do link acima contém mais linhas do que a seleção utilizada. Você pode utilizar todas as linhas ou considerar apenas uma fração dos dados. Na prática, quanto maior a quantidade de dados utilizada, mais confiável a análise! Mas existem limites computacionais e financeiros na qual a redução de dados para análise para fins de estudo se torna interessante.

Como são os dados

Query: **_SELECT * FROM credito LIMIT 10_** ;Dez primeiras linhas do dataset

É possível reparar que existem algumas informações nulas na tabela (valor na), vamos olhar mais de perto para os valores de cada coluna!

Quais os tipos de cada dado

Query: **_DESCRIBE credito_**

Descrição dos tipos de dados

Agora que ja entendemos quais são os tipos de dados, vamos olhar mais atentamente para as varíaveis que não são numéricas.

Quais são os tipos de escolaridade disponíveis no dataset?

Query: **SELECT DISTINCT escolaridade FROM credito**
    
Tipos de escolaridade

Os dados contam com vários níveis de escolaridade e é possível perceber que temos valores nulos (na) no dataset, trataremos isso mais tarde!

Qual a pocentagem da escolaridade, tipo de carão e o limite do cartão disponíveis no dataset?

Query: **SELECT DISTINCT escolaridade, tipo_cartao, limite_cartao FROM credito**
    
Será que temos mais valores nulos?

Quais são os tipos de estado_civil disponíveis no dataset?

Query: SELECT DISTINCT estado_civil FROM credito

Tipos de estado civil

Novamente encontramos valores nulos nos dados de estado civil!

Quais são os tipos de salario_anual disponíveis no dataset?

Query: SELECT DISTINCT salario_anual FROM credito

Tipos de salario anual

Os salários nesse dataset não estão expostas com o valor exato que o cliente ganha. O que está informado é a faixa salarial de cada um. Também contem dados nulos.

Quais são os tipos de cartão disponíveis no dataset?

Query: SELECT DISTINCT tipo_cartao FROM credito

Tipos de cartão

Aqui vemos que não há necessidade de tratar valores nulos.

Análise de dados
Uma vez que exploramos o dados e buscamos entender quais são as informações que temos no nosso banco de dados, podemos analisar as informações para buscar entender o que está acontecendo no banco de dados. Vamos fazer perguntas: / Nesse banco de dados, quantos clientes temos de cada faixa salarial?

Query: select count(*), salario_anual from credito group by salario_anual

Quantidade para cada faixa salarial

a maioria dos clientes dessa base de dados possui um renda menor que 40K e existem 235 clientes que não informaram ou não consta a faixa salarial. De certa forma, pode ser interessante para a empresa focar nesse público de mais baixa renda.

Nesse banco de dados, quantos clientes são homens e quantos são mulheres?

Query: select count(*), sexo from credito group by sexo

Quantidade para cada sexo

Quantidade para cada sexo - gráfico

A maioria dos clientes desse banco é homem! Do csv extraído dos dados é possível gerar o gráfico em pizza com para melhor a proporção de masculino/feminino

Queremos focar o nosso marketing de maneira adequada para nossos clientes, qual será a idade deles?

Query: select avg(idade) as media_idade, min(idade) as min_idade, max(idade) as max_idade, sexo from credito group by sexoMédia de idades por sexo

Por meio dessa análise não foi possível extrair nenhuma informação relevante. A menor idade dos dois sexos é a mesma e a média é muito similar. A unica diferença é a idade máxima mas fato quase irrelevante por a diferença não é tão gritante.

Qual a maior e menor transação dos clientes?

Query: select min(valor_transacoes_12m) as transacao_minima, max(valor_transacoes_12m) as transacao_minima from creditoValor transacoes

Nesse banco de dados temos soma de transações em 12 meses variam de 510.16 a 5776.58

Quais as características dos clientes que possuem os maiores creditos?

Query: select max(limite_credito) as limite_credito, escolaridade, tipo_cartao, sexo from credito where escolaridade != 'na' and tipo_cartao != 'na' group by escolaridade, tipo_cartao, sexo order by limite_credito desc limit 10

Valor limite

Não parece haver um impacto da escolaridade no limite. O limite mais alto é oferecido para um homem sem educação formal. O cartão também parece não estar relacionado com a escolaridade nem com o limite. Dentre os maiores limites, encontramos clientes com cartão: gold, silver, platinum e blue

Quais as características dos clientes que possuem os menores creditos?

Query: select max(limite_credito) as limite_credito, escolaridade, tipo_cartao, sexo from credito where escolaridade != 'na' and tipo_cartao != 'na' group by escolaridade, tipo_cartao, sexo order by limite_credito asc

Valor limite

Dessa vez conseguimos perceber que não há clientes com cartão platinum dentre os menores limites. Também foi possível perceber que a maioria dos menores limites são mulheres enquanto nos maiores limites predomina homens.

Será que as mulheres gastam mais?

Query: select max(valor_transacoes_12m) as maior_valor_gasto, avg(valor_transacoes_12m) as media_valor_gasto, min(valor_transacoes_12m) as min_valor_gasto, sexo from credito group by sexo

Valor transacoes/sexo

Apesar da diferença nos limites, os gastos de homens e mulheres são similares!

Por fim,

O salário impacta no limite?

Query: select avg(qtd_produtos) as qts_produtos, avg(valor_transacoes_12m) as media_valor_transacoes, avg(limite_credito) as media_limite, sexo, salario_anual from credito where salario_anual != 'na' group by sexo, salario_anual order by avg(valor_transacoes_12m) desc

Valor salario_anualLimite

SIM! As pessoas que tem menor faixa salarial também apresentam menor limite de credito!

Conclusão
Essas foram algumas análises extraídas do dataset de crédito.

Alguns insights interessantes:

a maior parte dos clientes possui renda até 40K
a maior parte dos clientes é masculino!
a escolaridade não parece influenciar no limite nem no tipo do cartão
os clientes na sua maioria tenhem como formaçao mestrado e a uma pocentagem considerada sem formação
os clientes com maiores limites são em sua maioria homens
os clientes com menores limites são em sua maioria mulheres
dentre os menores limites não há presença de cartão platinum
a faixa salarial impacta diretamente no limite de crédito
nao existem clientes com salário anual acima de 60K do sexo feminino
Uma exploração maior dos dados pode explicar porque as mulheres tem menor crédito. Isso também pode ser um problema cultural que pode ser repensado!